In [4]:
import xarray as xr
import numpy as np

In [104]:
array = xr.DataArray(np.random.randn(2, 3, 30), coords=[('x', ['a', 'b']), ('y', [0, 1, 2]), ('t',range(1,31))])

print(array)

<xarray.DataArray (x: 2, y: 3, t: 30)>
array([[[ 6.62854661e-01,  5.23286386e-01,  1.78582474e+00,
          1.59955592e+00,  1.19905536e+00,  7.15664181e-01,
          5.40991295e-02, -6.03258210e-01, -1.28670162e+00,
          1.23663453e+00, -1.04647509e+00,  9.21692669e-01,
         -7.46103555e-01,  1.08183315e+00,  2.23275932e-01,
         -1.93471976e+00,  5.82826532e-01,  2.51985292e+00,
         -1.29423520e+00,  3.10085214e-01, -6.75783077e-01,
         -8.92979131e-01,  7.35570757e-03, -2.23133383e-01,
         -5.15075477e-01, -3.95382417e-01, -3.54488012e-01,
         -1.29947847e+00, -6.59164630e-02,  7.98511929e-01],
        [-1.14068906e+00, -6.82954643e-01, -6.40100286e-01,
          1.97754622e+00, -1.07493346e+00, -2.23859364e+00,
         -7.81699833e-01,  6.13111220e-01, -1.02601300e-01,
          9.09747426e-01,  1.36707660e+00,  6.37863415e-01,
         -2.40244986e-01, -5.38564976e-01,  1.63618450e-01,
          5.56364050e-01, -1.93130733e-01,  8.50665101e-01,


Definition of dry spell
-----------------------
* A dry spell is a duration of 5 days or more, which receives a total rainfall less than 5mm
* Two consecutives dry spells are combined into one spell

E.g: (1, 0, 0, 0, 1, 0, 2, 0), 6 5 2 0 -----> one dry spell of length 7

E.g: (1, 0, 0, 0, 1, 0, 2, 0), (1, 0, 0, 1, 2), 6 5 2 0 -----> two consecutive dry spell of length 7 + dry spell of length 5 = dry spell of length 12

PROBLEM WITH THIS DEFINITON
---------------------------
 3,  2,  1,  0,  0,   2,  1,   2,  1,  0,   0,  0, 1

 3,  2, (1,  0,  0,   2,  1), (2,  1,  0,   0,  0, 1) ----> two consecutive dry spells = dry spell of length 11

3,  2, (1,  0,  0,   2,  1,   2,  1,  0,   0,  0, 1)

PROPOSITION
-----------
* A dry spell is a duration of 5 days or more which receives an average rainfall <1mm per day

3,  2,  (1,  0,  0,  2,  1),   2,  1,  0,   0,  0, 1 --> one dry spell of 5 days


In [28]:
import xarray as xr
import numpy as np
import pdb
array = xr.DataArray(np.random.normal(20,15,[2, 3, 30]), coords=[('x', ['a', 'b']), ('y', [0, 1, 2]), ('t',range(1,31))])
array.name = 'precip'
stacked = array.stack(z=('x', 'y'))
stacked_old = stacked.copy()

def precip_of_spell(t1, t2, xy):
    spell = stacked.loc[dict(t=slice(t1, t2), z=xy)]
    return spell.sum().values

### DRY SPELL COMBINED CONSECUTIVES
for xy in stacked.z:
    time_lead = 1
    spell_thresh = 2
    prec_thresh = 20    
    while time_lead < 31:
        long_dry_spell = 0
        end_of_spell = False
        new_time_lead = time_lead
        while not end_of_spell:
            wet = False
            dry_spell = 0
            
            while new_time_lead+dry_spell < 30 and not wet:
                if precip_of_spell(new_time_lead, new_time_lead+dry_spell, xy) < prec_thresh:
                    dry_spell += 1
                else:
                    wet = True
            long_dry_spell += dry_spell
            new_time_lead = time_lead + long_dry_spell
            if new_time_lead + spell_thresh-1 >= 30:
                end_of_spell = True        
            elif precip_of_spell(new_time_lead, new_time_lead+spell_thresh-1, xy) >= prec_thresh:
                end_of_spell = True 
                
        if long_dry_spell < spell_thresh:
            stacked.loc[dict(t=time_lead, z=xy)] = 0
            time_lead += 1
        else:
            for tt in range(time_lead, time_lead+long_dry_spell+1):
                stacked.loc[dict(t=tt, z=xy)] = long_dry_spell
            time_lead += long_dry_spell
            
            
            
print(stacked.loc[dict(z=('a',0))].values)
print(stacked_old.loc[dict(z=('a',0))].values)
print(stacked.loc[dict(t=slice(1, 10), z=('a',0))])
print(stacked)

KeyboardInterrupt: 

In [25]:
import xarray as xr
import numpy as np
import pdb
array = xr.DataArray(np.random.normal(20,15,[2, 3, 30]), coords=[('x', ['a', 'b']), ('y', [0, 1, 2]), ('t',range(1,31))])
array.name = 'precip'
stacked = array.stack(z=('x', 'y'))
stacked_old = stacked.copy()

def precip_of_spell(t1, t2, xy):
    spell = stacked.loc[dict(t=slice(t1, t2), z=xy)]
    return spell.sum().values

### DRY SPELL UNCOMBINED CONSECUTIVES
for xy in stacked.z:
    time_lead = 1
    spell_thresh = 2
    prec_thresh = 15    
    while time_lead < 31:
        wet = False
        dry_spell = 0
        while time_lead+dry_spell < 30 and not wet:
            if precip_of_spell(time_lead, time_lead+dry_spell, xy) < prec_thresh:
                dry_spell += 1
            else:
                wet = True
        if dry_spell < spell_thresh:
            stacked.loc[dict(t=time_lead, z=xy)] = 0
            time_lead+=1
        if dry_spell >= spell_thresh:
            for tt in range(time_lead, time_lead+dry_spell+1):
                stacked.loc[dict(t=tt, z=xy)] = dry_spell
            time_lead += dry_spell
print(stacked.loc[dict(z=('a',0))].values)
print(stacked_old.loc[dict(z=('a',0))].values)
print(stacked.loc[dict(t=slice(1, 10), z=('a',0))])
print(stacked)

[0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 4. 4. 4. 4. 2. 2. 2. 2. 2. 2. 0. 0. 2. 2.
 2. 2. 0. 0. 0. 0.]
[ 19.35261176  18.09249432  26.80606609  36.4334627   40.83413771
  18.34613625  -7.50764199  -1.2576236   44.09945399  26.82164997
  -0.48205827   8.05127155 -24.22387094   0.28477943  38.64043087
   0.92214123  26.44921555  10.16125627  27.70190691  12.29493205
  30.8998565   21.17892256 -13.10497436  12.43597062  26.96708081
   7.85703191  21.25726224  18.64053152  12.29679205  17.03261456]
<xarray.DataArray 'precip' (t: 10)>
array([0., 0., 0., 0., 0., 0., 2., 2., 0., 0.])
Coordinates:
  * t        (t) int64 1 2 3 4 5 6 7 8 9 10
    z        object ('a', 0)
<xarray.DataArray 'precip' (t: 30, z: 6)>
array([[0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 5.],
       [0., 0., 0., 0., 0., 5.],
       [0., 0., 0., 0., 0., 5.],
       [0., 0., 0., 0., 0., 5.],
       [2., 0., 2., 0., 0., 5.],
       [2., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
 

In [4]:
stacked.sel(t=time_lead, z=xy)
    

<xarray.DataArray 'precip' ()>
array(25.78926152)
Coordinates:
    t        int64 1
    z        object ('a', 0)

In [14]:
print(stacked.z)

<xarray.DataArray 'z' (z: 6)>
array([('a', 0), ('a', 1), ('a', 2), ('b', 0), ('b', 1), ('b', 2)],
      dtype=object)
Coordinates:
  * z        (z) MultiIndex
  - x        (z) object 'a' 'a' 'a' 'b' 'b' 'b'
  - y        (z) int64 0 1 2 0 1 2


In [29]:
array.groupby('t')
threshold = 1
array.where(array < 1, 1)                                      
array = array.where(array > 1, 0)
print(array)

<xarray.DataArray (x: 2, y: 3, t: 3)>
array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [1.77396307, 0.        , 0.        ]],

       [[1.24981969, 0.        , 0.        ],
        [0.        , 0.        , 1.79621743],
        [2.08529255, 0.        , 0.        ]]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 0 1 2
  * t        (t) int64 2000 2001 2002


In [2]:
import xarray as xr
import numpy as np
import pdb
array = xr.DataArray(np.random.normal(20,15,[2, 3, 10]), coords=[('x', ['a', 'b']), ('y', [0, 1, 2]), ('t',range(1,11))])
array.name = 'precip'
stacked = array.stack(z=('x', 'y'))
stacked_old = stacked.copy()

In [3]:
def precip_of_spell(t1, t2, xy):
    spell = stacked.loc[dict(t=slice(t1, t2), z=xy)]
    return spell.sum().values

### DRY SPELL COMBINED CONSECUTIVES
for xy in stacked.z:
    time_lead = 1
    spell_thresh = 3
    prec_thresh = 30
    while time_lead < 11:
        long_dry_spell = 0
        end_of_spell = False
        new_time_lead = time_lead
        while not end_of_spell:
            wet = False
            dry_spell = 0
            
            while new_time_lead+dry_spell < 10 and not wet:
                if precip_of_spell(new_time_lead, new_time_lead+dry_spell, xy) < prec_thresh:
                    dry_spell += 1
                else:
                    wet = True
            long_dry_spell += dry_spell
            new_time_lead = time_lead + long_dry_spell
            if not wet:
                end_of_spell = True        
            elif precip_of_spell(new_time_lead, new_time_lead+spell_thresh-1, xy) >= prec_thresh:
                end_of_spell = True 
                
        if long_dry_spell < spell_thresh:
            stacked.loc[dict(t=time_lead, z=xy)] = 0
            time_lead += 1
        else:
            for tt in range(time_lead, time_lead+long_dry_spell+1):
                stacked.loc[dict(t=tt, z=xy)] = long_dry_spell
            time_lead += long_dry_spell
            
            
            
print(stacked.loc[dict(z=('a',0))].values)
print(stacked_old.loc[dict(z=('a',0))].values)
print(stacked.loc[dict(t=slice(1, 10), z=('a',0))])
print(stacked)

KeyboardInterrupt: 